In [1]:
// import SparkSession
import org.apache.spark.sql.SparkSession

// create a SparkSession
val spark = SparkSession.builder()
                        .appName("Graph Creation")
                        .master("local[*]")
                        .getOrCreate()

// create an RDD for the edges
var edges_rdd = spark.sparkContext.parallelize(List((0,1),(1,2),(1,3),(3,4),(5,6),(6,7),(7,8)))

Intitializing Scala interpreter ...

Spark Web UI available at http://mbp-de-alicia-2.lan:4043
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1680451211657)
SparkSession available as 'spark'


23/04/02 18:00:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7456dbf1
edges_rdd: org.apache.spark.rdd.RDD[(Int, Int)] = ParallelCollectionRDD[0] at parallelize at <console>:33


In [2]:
// Function for the reducer

def reducer(edges: (Int, List[Int])): (List[(Int, Int)], Int) = {
    val key = edges._1
    val values = edges._2
    var minim = key
    var valueList = List.empty[Int]
    var to_emit = List.empty[(Int, Int)]
    var counter = 0
    for (value <- values) {
        if (value < minim) {
            minim = value
        }
        valueList = valueList :+ value
    }
    if (minim < key) {
        to_emit = (key, minim) :: to_emit
        for (value <- valueList) {
            if (minim != value) {
                counter += 1
                to_emit = (value, minim) :: to_emit
            }
        }
    }
    (to_emit, counter)
}

reducer: (edges: (Int, List[Int]))(List[(Int, Int)], Int)


In [3]:
import scala.collection.mutable.ListBuffer
import scala.util.control.Breaks._

var count_new_pairs = 1
var iterations = 0

while (count_new_pairs > 0) {
  val result_map = edges_rdd.flatMap(x => List((x._1, x._2), (x._2, x._1)))
  val result_shufflesort = result_map.groupByKey().map(x => (x._1, x._2.toList))
  val result_reducer = result_shufflesort.map(x => reducer(x))
  val result_reducer2 = result_reducer.flatMap(x => x._1)
  count_new_pairs = result_reducer.map(x => x._2).reduce((x, y) => x + y)
  edges_rdd = result_reducer2.distinct()
  iterations += 1
}

val sorted_rdd = edges_rdd.sortBy(x => x._2)
println(sorted_rdd.collect().toList)
println("Iterations: " + iterations)

List((4,0), (1,0), (3,0), (2,0), (7,5), (8,5), (6,5))
Iterations: 4


import scala.collection.mutable.ListBuffer
import scala.util.control.Breaks._
count_new_pairs: Int = 0
iterations: Int = 4
sorted_rdd: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[41] at sortBy at <console>:42
